# Project 7. Ford vs. Ferrari
# Классификация изображений автомобилей

Основная идея этого решения: взять предобученую на ImageNet сеть Xception и дообучить под нашу задачу. 
Постараться улучшить решение, приведенное в бейзлайне, используя известные инструменты.

In [1]:
!pip show  -q
!pip install -q efficientnet
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor -q

In [2]:
!nvidia-smi

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *
from keras.models import Sequential
from ImageDataAugmentor.image_data_augmentor import *
import albumentations
from tensorflow.keras.backend import clear_session
from keras.applications import *
import efficientnet.tfkeras as efn
from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [4]:
!pip freeze > requirements.txt

# Основные настройки

In [5]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 10  # эпох на обучение
BATCH_SIZE           = 64 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-3
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/'
PATH = "../working/car/" # рабочая директория

In [6]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH,exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  

# EDA / Анализ данных

In [7]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [8]:
train_df.info()

В датасете нет пропусков.

In [9]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное

In [10]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [11]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [12]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

Классы в данной задаче представляют собой модели автомобилей.

In [13]:
print('Пример картинок по категориям')
plt.figure(figsize=(12,8))


for i in range(10):
    image = train_df[train_df['Category'] == i].head(9)
    image_paths = image['Id'].values
    image_cat = image['Category'].values

    for index, path in enumerate(image_paths):
        im = PIL.Image.open(PATH + f'train/{image_cat[index]}/{path}')
        plt.subplot(3,3, index+1)
        plt.imshow(im)
        plt.title('Class: '+str(image_cat[index]))
        plt.axis('off')
    plt.show()

# Подготовка данных

### Аугментация данных

Способ аугментации из бейзлайна показал качество хуже в сравнении с применением albuminations, не будем его использовать.

In [14]:
# Выполним аугментацию данных для лучшего обучения модели


# train_datagen = ImageDataGenerator(
#    rescale=1. / 255,
#    rotation_range = 5,
#    width_shift_range = 0.1,
#    height_shift_range = 0.1,
#    validation_split = VAL_SPLIT, # set validation split
#    horizontal_flip = False,
#    fill_mode = 'nearest',
#    shear_range = 0.1,
#    zoom_range = 0.1)

               
                                                                                                                      
# test_datagen = ImageDataGenerator(rescale=1. / 255)

In [15]:
augments = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, 
                          interpolation=1, 
                          border_mode=4, 
                          value=None, 
                          mask_value=None, 
                          always_apply=False, 
                          p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, width=200),
        albumentations.RandomCrop(height=200, width=224),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])             

train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = augments,
        validation_split = VAL_SPLIT,
        seed = RANDOM_SEED
        )

test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  validation_split = VAL_SPLIT,
                                  # seed = RANDOM_SEED
        )

sub_datagen = ImageDataGenerator(
        rescale=1./255,    
    # seed = RANDOM_SEED
        )

### Генерация данных

In [16]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PATH +'train/',      # директория где расположены папки с картинками 
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = True, seed=RANDOM_SEED,
    subset = 'training') # set as training data

test_generator = test_datagen.flow_from_directory(
    PATH +'train/',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle = True, seed = RANDOM_SEED,
    subset = 'validation') # set as validation data

test_sub_generator = sub_datagen.flow_from_dataframe( 
    dataframe = sample_submission,
    directory = PATH +'test_upload/',
    x_col = "Id",
    y_col = None,
    shuffle = False,
    class_mode = None,
    seed = RANDOM_SEED,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,)

In [17]:
from skimage import io

def imshow(image_RGB):
    io.imshow(image_RGB)
    io.show()

x,y = train_generator.next()
print('Пример картинок из train_generator')
plt.figure(figsize=(12,8))

for i in range(0,9):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
    #plt.title('Class: '+str(y[i]))
    #plt.axis('off')
plt.show()

# Построение модели

### Загружаем предобученную сеть Xception:

In [18]:
clear_session()

In [19]:
# Загрузим предобученную сеть Xception
base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

In [20]:
# Заморозим слои базовой модели для последующего Fine-tuning
# base_model.trainable = False

In [21]:
base_model.summary()

In [22]:
# Устанавливаем новую "голову" (head)
# пробуем добавить batch-нормализацию (не дало прироста качества)

x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = BatchNormalization()(x)
# let's add a fully-connected layer
x = Dense(256, activation='relu')(x)
# x = BatchNormalization()(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 10 classes
predictions = Dense(CLASS_NUM, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), 
              metrics=["accuracy"])

In [23]:
model.summary()

### Возьмем для сравнения сеть EfficientNetB5

In [24]:
# base_model = efn.EfficientNetB5(weights='imagenet', include_top=False, input_shape=input_shape)

In [25]:
# base_model.trainable = False

In [26]:
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = BatchNormalization()(x)
# x = Dropout(0.25)(x)
# x = Dense(256, activation = 'relu')(x)
# x = BatchNormalization()(x)
# predictions = Dense(10, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)
# model.compile(loss="categorical_crossentropy", optimizer = optimizers.Adam(learning_rate = LR), 
#             metrics = 'accuracy')

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [27]:
checkpoint = ModelCheckpoint('best_model.hdf5', monitor = ['val_accuracy'], 
                             verbose = 1, mode = 'max')
earlystop = EarlyStopping(monitor='accuracy', patience=5, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

In [28]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [29]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [30]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики обучения:

In [31]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

### Fine-tuning

#### 1. Разморозка половины слоев

In [32]:
# Определим количество слоев в базовой модели
len(base_model.layers)

In [33]:
# Укажем, что будем обучать базовую модель
base_model.trainable = True

for layer in base_model.layers[:65]:
    layer.trainable = False

In [34]:
len(base_model.trainable_variables)

In [35]:
model.compile(loss="categorical_crossentropy", optimizer = optimizers.Adam(learning_rate = LR), 
              metrics = 'accuracy')

In [36]:
model.summary()

In [37]:
history = model.fit(
    train_generator, 
    steps_per_epoch = len(train_generator), 
    validation_data = test_generator, 
    validation_steps = len(test_generator), 
    epochs = EPOCHS, 
    callbacks = callbacks_list
)

In [38]:
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [39]:
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [40]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

#### Разморозим больше слоев

In [41]:
base_model.trainable = True

for layer in base_model.layers[:30]:
    layer.trainable = False
    
len(base_model.trainable_variables)    

In [42]:
model.compile(loss="categorical_crossentropy", optimizer = optimizers.Adam(learning_rate = LR), 
              metrics = 'accuracy')
model.summary()

In [43]:
history = model.fit(
    train_generator, 
    steps_per_epoch = len(train_generator), 
    validation_data = test_generator, 
    validation_steps = len(test_generator), 
    epochs = EPOCHS, 
    callbacks = callbacks_list
)

In [44]:
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [45]:
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [46]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

#### Разморозим все слои

In [47]:
base_model.trainable = True


In [48]:
model.compile(loss="categorical_crossentropy", optimizer = optimizers.Adam(learning_rate = LR), 
              metrics = 'accuracy')

In [49]:
history = model.fit(
    train_generator, 
    steps_per_epoch = len(train_generator), 
    validation_data = test_generator, 
    validation_steps = len(test_generator), 
    epochs = 15, 
    callbacks = callbacks_list
)

In [50]:
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [51]:
scores = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [52]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# Предсказание на тестовых данных

In [53]:
test_sub_generator.samples

In [54]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [55]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

# Рекомендация: попробуйте добавить Test Time Augmentation (TTA)
# https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [56]:
submission.head()

In [57]:
# Clean PATH
import shutil
shutil.rmtree(PATH)